In [1]:
from imblearn.over_sampling import ADASYN
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
data=np.load('PDNA224_HHM_7.npz')
row, col = 15, 30
x_pos, x_neg = data['pos'], data['neg']
x_neg = x_neg.reshape(-1,row*col)
x_pos = x_pos.reshape(-1,row*col)

In [3]:
n_splits = 10
kf = KFold(n_splits=n_splits)
kf_x_pos_train, kf_x_pos_test = [],[]
for train_index, test_index in kf.split(x_pos):
    kf_x_pos_train.append(x_pos[train_index])
    kf_x_pos_test.append(x_pos[test_index])

In [11]:
kf_x_neg_train, kf_x_neg_test = [],[]
for train_index, test_index in kf.split(x_neg):
    kf_x_neg_train.append(x_neg[train_index])
    kf_x_neg_test.append(x_neg[test_index])

In [12]:
adasyn = ADASYN()
kf_x_train, kf_y_train = [],[]
kf_x_test, kf_y_test = [],[]
for i in range(n_splits):
    print(i)
    x = np.concatenate((kf_x_pos_train[i], kf_x_neg_train[i]))
    y = np.zeros( (len(x),))
    y[len(kf_x_pos_train[i]):] = 1
    x_as, y_as = adasyn.fit_resample(x,y)
    x_as = x_as.reshape(-1, row, col)
    y_as = tf.keras.utils.to_categorical(y_as, num_classes=2)
    kf_x_train.append(x_as)
    kf_y_train.append(y_as)
    
    x = np.concatenate((kf_x_pos_test[i], kf_x_neg_test[i]))
    y = np.zeros((len(x),))
    y[len(kf_x_pos_test[i]):] = 1
    y = tf.keras.utils.to_categorical(y, num_classes=2)
    x = x.reshape(-1, row, col)
    kf_x_test.append(x)
    kf_y_test.append(y)

np.savez('PDNA224_HHM_7_kfold10_resampling.npz',x_train=kf_x_train, 
        y_train=kf_y_train, x_test=kf_x_test, y_test=kf_y_test)

0
1
2
3
4
5
6
7
8
9
